In [1]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
import model as mod
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
import dataset
from dataset import InputFeatures, load_and_cache_examples
import functionsAval as f
from sklearn_crfsuite.metrics import flat_f1_score, flat_classification_report
import joblib
import os


In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
def has_numbers(inputString):
    return str(bool(re.search(r'\d', inputString)))

print(has_numbers('metionina 5mg'))

def isstopword(p):
  return str(p in stopwords.words('portuguese'))

print(isstopword('pelo'))


True
True


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lisat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def powerset(entidade):    
    lista=list()
    for i in range(1, len(entidade)+1):
        #lista.append(entidade[i-1])
        for j in range(len(entidade) - i + 1):
            lista.append(entidade[j:j + i])
    return lista
        
#print(list(powerset([4, 5, 6])))
print(powerset(['dor', 'no', 'peito']))

[['dor'], ['no'], ['peito'], ['dor', 'no'], ['no', 'peito'], ['dor', 'no', 'peito']]


In [2]:
f = reload(f)
reload(dataset)
reload(eval)
reload(mod)

# em numero de frases
BATCH=506
#BATCH=5
#BATCH=800
#BATCH=8000 
print('BATCH:', BATCH)

BATCH: 506


In [3]:
dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())
#print(sentences[0])

tags, tokens = f.predictBERTNER_IO(sentences, 'all')
dic_predictions = f.getDicPredictions(tags, tokens)
dic_predictions_ner_bert = f.getDicPredictions(tags, tokens)
print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))
#print(dic_predictions[9])
f.save_obj('dic_predictions_results_ner_'+str(BATCH), dic_predictions)
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))

print('len(dicSentences_new_test):', len(dicSentences_new_test))
print('len(dic_predictions):', len(dic_predictions))

region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)
f.save_obj('region_true_list'+str(BATCH), region_true_list)
print('region_pred_list[:4]:', region_pred_list[:4])
print('region_true_list[:4]:', region_true_list[:4])
print('lista_erros[:8]:', lista_erros[:8])
print('len(lista_erros):', len(lista_erros))
print('len(set(lista_erros)):', len(set(lista_erros)))
print('len(region_true_list):', len(region_true_list))
print('len(region_pred_list):', len(region_pred_list))

print('-----Avaliando nested só com modelo de NER com BERT:-----')

print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))


Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Lucas', 0, 43], [',', 1, 48], ['74', 2, 50], ['anos', 3, 53], ['.', 4, 57]], []]
numero de sentencas no total: 506
idx2tag: {0: 'Teste', 1: 'Anatomia', 2: 'O', 3: 'Problema', 4: 'Tratamento', 5: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506
len(dicSentences_new_test): 506
len(dic_predictions): 506
region_pred_list[:4]: ['Problema', 'Tratamento', 'Problema', 'Problema']
region_true_list[:4]: ['Problema', 'Tratamento', 'Problema', 'Problema']
lista_erros[:8]: [7, 8, 13, 13, 14, 15, 15, 15]
len(lista_erros): 383
len(set(lista_erros)): 169
len(region_true_list): 1140
len(region_pred_list): 1140
-----Avaliando nested só com modelo de NER com BERT:-----
              precision    recall  f1-score   support

    Anatomia   0.829268  0.346939  0.489209       196
           O   0.000000  0.000000  0.000000       149
    Problema   0.798319  0.843195  0.820144       338
      

In [4]:
tags[0]

['O', 'O', 'O', 'O', 'O']

In [5]:
tokens[0]

['Lucas', ',', '74', 'anos', '.']

### AVAL spans / regioes

In [11]:
dic_sentencesTest = dicSentences_new_test.copy()
dic_sentencesTest[2]

[[['Comorbidades', 0, 142],
  [':', 1, 154],
  ['DM', 2, 156],
  ['há', 3, 159],
  ['10', 4, 162],
  ['anos', 5, 165],
  ['em', 6, 170],
  ['uso', 7, 173],
  ['de', 8, 177],
  ['metformina', 9, 180],
  ['850mg', 10, 191],
  ['3', 11, 197],
  ['cp', 12, 199],
  ['/', 13, 201],
  ['dia', 14, 202],
  [',', 15, 205],
  ['acarbose', 16, 207],
  ['1', 17, 216],
  ['cp', 18, 218],
  ['/', 19, 220],
  ['dia', 20, 221],
  ['e', 21, 225],
  ['glicazida', 22, 227],
  ['60mg', 23, 237],
  ['2', 24, 242],
  ['cp', 25, 244],
  ['/', 26, 246],
  ['dia', 27, 247],
  ['e', 28, 251],
  ['insulina', 29, 253],
  ['(', 30, 262],
  ['24', 31, 263],
  ['-', 32, 266],
  ['0', 33, 268],
  ['-', 34, 270],
  ['24', 35, 272],
  [')', 36, 274],
  ['.', 37, 275]],
 [['Comorbidades', [0], 'Problema'],
  ['DM', [2], 'Problema'],
  ['metformina 850mg', [9, 10], 'Tratamento'],
  ['acarbose', [16], 'Tratamento'],
  ['glicazida 60mg', [22, 23], 'Tratamento'],
  ['insulina', [29], 'Tratamento']]]

In [18]:
NUM_JANELA=4

#def getListaPositiva(dic_sentences):
#dic_sentences = dic_sentencesTest
def getListaCombinacoesOld(dic_sentences):
    listaSentencas=list()
    lista=list()
    numJanela=NUM_JANELA
    for key, value in dic_sentences.items():
        #print('value:', value)
        entidades = value[1]
        if len(entidades)==0:
            continue
        listaIndicesEntidadesFrase=list()
        for entidade in entidades:
          listaIndicesEntidadesFrase.append(entidade[1])
        dicTokens={}
        for token in value[0]:
          #print('token:', token)
          dicTokens[token[1]]=token[0]
        for entidade in entidades:
            #print('entidade:', entidade)
            label = entidade[2]
            indiceEntidade1=entidade[1][0]
            indiceEntidade2=entidade[1][-1]
            vizinhosAntes = list()
            vizinhosDepois = list()
            #print('indiceEntidade:', indiceEntidade)
            for tokens in value[0]:
                indice=tokens[1]
                #print('token: {}, indice: {}'.format(tokens[0], indice))
                if indice+4>=indiceEntidade1 and indice<indiceEntidade1:
                    vizinhosAntes.append(tokens[0])
                if indice-4<=indiceEntidade2 and indice>indiceEntidade2:
                    vizinhosDepois.append(tokens[0])
            lista.append([entidade[0], vizinhosAntes, vizinhosDepois, label])
            # agora, os negativos, para entidades com mais de um token
            if len(entidade[0].split())>1:
              #print('entidade[0] mais de um token: {}'.format(entidade[0].split()))
              #listaCombinacoes = powerset(entidade[0].split())
              listaCombinacoes = powerset(entidade[1])
              #print(listaCombinacoes)
              # ver se alguma dessas é positiva
              for combinacao in listaCombinacoes:
                #print('combinacao:', combinacao)
                if combinacao in listaIndicesEntidadesFrase:
                  #print('teeeem:', combinacao)
                  pass
                else:
                  combinacaoTokens = [dicTokens[t] for t in combinacao]
                  vizinhosAntes = list()
                  vizinhosDepois = list()
                  indiceEntidade1=combinacao[0]
                  indiceEntidade2=combinacao[-1]
                  for tokens in value[0]:
                    indice=tokens[1]
                    #print('token: {}, indice: {}'.format(tokens[0], indice))
                    if indice+4>=indiceEntidade1 and indice<indiceEntidade1:
                        vizinhosAntes.append(tokens[0])
                    if indice-4<=indiceEntidade2 and indice>indiceEntidade2:
                        vizinhosDepois.append(tokens[0])
                  lista.append([' '.join(combinacaoTokens), vizinhosAntes, vizinhosDepois, 'O'])
        listaSentencas.append(lista)
        lista=list()
        #break
    return listaSentencas
listacombinacoes = getListaCombinacoesOld(dic_sentencesTest)
print(listacombinacoes[:2])

[[['FA', ['há', '5', 'anos', 'por'], [',', 'uso', 'de', 'marevan'], 'Problema'], ['marevan 5mg', ['FA', ',', 'uso', 'de'], ['1', 'x', 'ao', 'dia'], 'Tratamento'], ['marevan', ['FA', ',', 'uso', 'de'], ['5mg', '1', 'x', 'ao'], 'O'], ['5mg', [',', 'uso', 'de', 'marevan'], ['1', 'x', 'ao', 'dia'], 'O']], [['Comorbidades', [], [':', 'DM', 'há', '10'], 'Problema'], ['DM', ['Comorbidades', ':'], ['há', '10', 'anos', 'em'], 'Problema'], ['metformina 850mg', ['anos', 'em', 'uso', 'de'], ['3', 'cp', '/', 'dia'], 'Tratamento'], ['metformina', ['anos', 'em', 'uso', 'de'], ['850mg', '3', 'cp', '/'], 'O'], ['850mg', ['em', 'uso', 'de', 'metformina'], ['3', 'cp', '/', 'dia'], 'O'], ['acarbose', ['cp', '/', 'dia', ','], ['1', 'cp', '/', 'dia'], 'Tratamento'], ['glicazida 60mg', ['cp', '/', 'dia', 'e'], ['2', 'cp', '/', 'dia'], 'Tratamento'], ['glicazida', ['cp', '/', 'dia', 'e'], ['60mg', '2', 'cp', '/'], 'O'], ['60mg', ['/', 'dia', 'e', 'glicazida'], ['2', 'cp', '/', 'dia'], 'O'], ['insulina', ['cp'

In [13]:
dic_predictions[2]

[[['Comorbidades', 0],
  [':', 1],
  ['DM', 2],
  ['há', 3],
  ['10', 4],
  ['anos', 5],
  ['em', 6],
  ['uso', 7],
  ['de', 8],
  ['metformina', 9],
  ['850mg', 10],
  ['3', 11],
  ['cp', 12],
  ['/', 13],
  ['dia', 14],
  [',', 15],
  ['acarbose', 16],
  ['1', 17],
  ['cp', 18],
  ['/', 19],
  ['dia', 20],
  ['e', 21],
  ['glicazida', 22],
  ['60mg', 23],
  ['2', 24],
  ['cp', 25],
  ['/', 26],
  ['dia', 27],
  ['e', 28],
  ['insulina', 29],
  ['(', 30],
  ['24', 31],
  ['-', 32],
  ['0', 33],
  ['-', 34],
  ['24', 35],
  [')', 36],
  ['.', 37]],
 [['Comorbidades', [0], 'Problema'],
  ['DM', [2], 'Problema'],
  ['metformina 850mg', [9, 10], 'Tratamento'],
  ['acarbose', [16], 'Tratamento'],
  ['glicazida 60mg', [22, 23], 'Tratamento'],
  ['insulina', [29], 'Tratamento']]]

In [26]:
#def getListaPositiva(dic_sentences):
#dic_sentences = dic_sentencesTest
def getListaCombinacoes(dic_sentences):
    listaSentencas=list()
    lista=list()
    dicPred={}
    #numJanela=NUM_JANELA
    numJanela=4
    num1=-1
    num2=0
    for key, value in dic_sentences.items():
        num1=num1+1
        #print('value:', value)
        entidades = value[1]
        if len(entidades)==0:
            dicPred[num2]=num1
            continue
        dicPred[num2]=num1
        num2=num2+1
        listaIndicesEntidadesFrase=list()
        for entidade in entidades:
          listaIndicesEntidadesFrase.append(entidade[1])
        dicTokens={}
        for token in value[0]:
          #print('token:', token)
          dicTokens[token[1]]=token[0]
        for entidade in entidades:
            #print('entidade:', entidade)
            #label = entidade[2]
            label = 'ENT'
            indiceEntidade1=entidade[1][0]
            indiceEntidade2=entidade[1][-1]
            vizinhosAntes = list()
            vizinhosDepois = list()
            #print('indiceEntidade:', indiceEntidade)
            for tokens in value[0]:
                indice=tokens[1]
                #print('token: {}, indice: {}'.format(tokens[0], indice))
                if indice+numJanela>=indiceEntidade1 and indice<indiceEntidade1:
                    vizinhosAntes.append(tokens[0])
                if indice-numJanela<=indiceEntidade2 and indice>indiceEntidade2:
                    vizinhosDepois.append(tokens[0])
            lista.append([entidade[0], vizinhosAntes, vizinhosDepois, label])
            # agora, os negativos, para entidades com mais de um token
            if len(entidade[0].split())>1:
              #print('entidade[0] mais de um token: {}'.format(entidade[0].split()))
              #listaCombinacoes = powerset(entidade[0].split())
              listaCombinacoes = powerset(entidade[1])
              #print(listaCombinacoes)
              # ver se alguma dessas é positiva
              for combinacao in listaCombinacoes:
                #print('combinacao:', combinacao)
                if combinacao in listaIndicesEntidadesFrase:
                  #print('teeeem:', combinacao)
                  pass
                else:
                  combinacaoTokens = [dicTokens[t] for t in combinacao]
                  vizinhosAntes = list()
                  vizinhosDepois = list()
                  indiceEntidade1=combinacao[0]
                  indiceEntidade2=combinacao[-1]
                  for tokens in value[0]:
                    indice=tokens[1]
                    #print('token: {}, indice: {}'.format(tokens[0], indice))
                    if indice+numJanela>=indiceEntidade1 and indice<indiceEntidade1:
                        vizinhosAntes.append(tokens[0])
                    if indice-numJanela<=indiceEntidade2 and indice>indiceEntidade2:
                        vizinhosDepois.append(tokens[0])
                  lista.append([' '.join(combinacaoTokens), vizinhosAntes, vizinhosDepois, 'O'])
        listaSentencas.append(lista)
        lista=list()
        #break
    return listaSentencas, dicPred


In [27]:
listacombinacoesTest, dicPred = getListaCombinacoes(dic_sentencesTest)
listacombinacoesTest[0:2]

[[['FA', ['há', '5', 'anos', 'por'], [',', 'uso', 'de', 'marevan'], 'ENT'],
  ['marevan 5mg', ['FA', ',', 'uso', 'de'], ['1', 'x', 'ao', 'dia'], 'ENT'],
  ['marevan', ['FA', ',', 'uso', 'de'], ['5mg', '1', 'x', 'ao'], 'O'],
  ['5mg', [',', 'uso', 'de', 'marevan'], ['1', 'x', 'ao', 'dia'], 'O']],
 [['Comorbidades', [], [':', 'DM', 'há', '10'], 'ENT'],
  ['DM', ['Comorbidades', ':'], ['há', '10', 'anos', 'em'], 'ENT'],
  ['metformina 850mg',
   ['anos', 'em', 'uso', 'de'],
   ['3', 'cp', '/', 'dia'],
   'ENT'],
  ['metformina', ['anos', 'em', 'uso', 'de'], ['850mg', '3', 'cp', '/'], 'O'],
  ['850mg', ['em', 'uso', 'de', 'metformina'], ['3', 'cp', '/', 'dia'], 'O'],
  ['acarbose', ['cp', '/', 'dia', ','], ['1', 'cp', '/', 'dia'], 'ENT'],
  ['glicazida 60mg', ['cp', '/', 'dia', 'e'], ['2', 'cp', '/', 'dia'], 'ENT'],
  ['glicazida', ['cp', '/', 'dia', 'e'], ['60mg', '2', 'cp', '/'], 'O'],
  ['60mg', ['/', 'dia', 'e', 'glicazida'], ['2', 'cp', '/', 'dia'], 'O'],
  ['insulina', ['cp', '/', 'd

In [19]:
def getTiposEntidadeBinario():
    return ['O','ENT']

def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']
    
def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        print(line)
        print(num)
        raise
    return word2cluster


def word2features(sent, word2cluster, dicPostagger):
    try:
        features = list()
        entidades = sent[0]
        for entidade in entidades.split():
            postag = tipoPostaggerTokens(entidade, dicPostagger)
            features.extend([
            'bias',
            'word.lower=' + entidade.lower(),
            'word[-3:]=' + entidade[-3:],
            'word[:3]=' + entidade[:3],
            'word.isupper=%s' % entidade.isupper(),
            'word.istitle=%s' % entidade.istitle(),
            'word.isdigit=%s' % entidade.isdigit(),
            'word.cluster=%s' % word2cluster[entidade.lower()] if entidade.lower() in word2cluster else "0",
            'postag=' + postag
            ])
        # palavras anteriores
        vizinhosAntes = sent[1] 
        if len(vizinhosAntes)>0:
            for num, vizinhoAntes in enumerate(vizinhosAntes):
                word1 = vizinhoAntes
                postag1 =  tipoPostaggerTokens(vizinhoAntes, dicPostagger)
                features.extend([
                    '-'+str(num+1)+':word.lower=' + word1.lower(),
                    '-'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '-'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '-'+str(num+1)+':postag=' + postag1
                ])
        else:
            features.append('BOS')

        # próximas palavras
        vizinhosDepois = sent[2]
        if len(vizinhosDepois)>0:
            for num, vizinhoDepois in enumerate(vizinhosDepois):
                word1 = vizinhoDepois
                postag1 =  tipoPostaggerTokens(vizinhoDepois, dicPostagger)
                features.extend([
                    '+'+str(num+1)+':word.lower=' + word1.lower(),
                    '+'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '+'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '+'+str(num+1)+':postag=' + postag1
                ])
        else:
            features.append('EOS')
    except:
        print('sent:', sent)
        raise
    return features


def sent2features(lista, word2cluster, dicPostagger):
    #return word2features(lista, word2cluster, dicPostagger)
    return [word2features(lista[i], word2cluster, dicPostagger) for i in range(len(lista))]

def sent2labels(lista):
    #return [label for _, _, _, label in lista]
    return [label for _, _, _, label in lista]

#def sent2tokens(sent):
#    return [token for token, postag, label in sent]

#word2cluster = read_clusters(r"cluster/cluster-50.tsv")
#word2cluster = read_clusters(r"clusters/cluster-5.tsv")
#word2cluster = read_clusters(r"clusters/cluster-10.tsv")
#word2cluster = read_clusters(r"clusters/cluster-100.tsv")
#word2cluster = read_clusters(r"clusters/cluster-300.tsv")

In [28]:
import joblib
import os

CLUSTER=300
word2cluster = read_clusters(r"cluster/cluster-300.tsv")

X_test = [sent2features(s, word2cluster, dicPostagger) for s in listacombinacoesTest]
y_test = [sent2labels(s) for s in listacombinacoesTest]

print('regiao CRF binario, cluster 300, janela 4 => sem estar no pipeline')

OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_binario_regiao"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)
print(flat_classification_report(
    y_test, y_pred, labels=getTiposEntidadeBinario(), digits=3
))

regiao CRF binario, cluster 300, janela 4 => sem estar no pipeline
              precision    recall  f1-score   support

           O      0.948     0.944     0.946      2730
         ENT      0.848     0.859     0.853       991

    accuracy                          0.921      3721
   macro avg      0.898     0.901     0.900      3721
weighted avg      0.922     0.921     0.921      3721



In [147]:
dic_predictions_ner_bert[15]

[[['calcificação', 0],
  ['mitral', 1],
  ['e', 2],
  ['aórtica', 3],
  ['com', 4],
  ['refluxo', 5],
  ['leve', 6],
  ['.', 7]],
 [['calcificação', [0], 'Problema'],
  ['mitral', [1], 'Tratamento'],
  ['e', [2], 'Problema'],
  ['aórtica', [3], 'Anatomia'],
  ['refluxo leve', [5, 6], 'Problema']]]

In [146]:
listacombinacoesTestPred[13]

[['calcificação', [], ['mitral', 'e', 'aórtica', 'com'], 'ENT'],
 ['mitral', ['calcificação'], ['e', 'aórtica', 'com', 'refluxo'], 'ENT'],
 ['e',
  ['calcificação', 'mitral'],
  ['aórtica', 'com', 'refluxo', 'leve'],
  'ENT'],
 ['aórtica',
  ['calcificação', 'mitral', 'e'],
  ['com', 'refluxo', 'leve', '.'],
  'ENT'],
 ['refluxo leve', ['mitral', 'e', 'aórtica', 'com'], ['.'], 'ENT'],
 ['refluxo', ['mitral', 'e', 'aórtica', 'com'], ['leve', '.'], 'O'],
 ['leve', ['e', 'aórtica', 'com', 'refluxo'], ['.'], 'O']]

In [141]:
CLUSTER=300
word2cluster = read_clusters(r"cluster/cluster-300.tsv")
#dic_predictions = f.getDicPredictions(y_pred, allTokens)

print('agora, com pipeline... regiao CRF binario, cluster 300, janela 4 => com pipeline')

#listacombinacoesTestPred, dicPred = getListaCombinacoes(dic_predictions)
listacombinacoesTestPred, dicPred = getListaCombinacoes(dic_predictions_ner_bert)

X_test = [sent2features(s, word2cluster, dicPostagger) for s in listacombinacoesTestPred]
#y_test = [sent2labels(s) for s in listacombinacoesTest]

OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_binario_regiao"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)

dic_sentencesTest2={}
num=0
for key, value in dic_sentencesTest.items():
    if value[1]:
        dic_sentencesTest2[num] = dic_sentencesTest[key]
        num=num+1
        
        #dic_sentencesTest
    
regiao_true=list()
regiao_pred=list()

dicLabels={'O':'0', 'ENT':'1', 'Problema':'1', 'Tratamento':'1', 'Teste':'1', 'Anatomia':'1'}

num=0
for ent, label in zip(listacombinacoesTestPred, y_pred):
    #print(num)
    #print(ent)
    numKey=dicPred[num]
    listaTrue=dic_sentencesTest[numKey][1]
    num=num+1
    #print(listaTrue)
    for entidade in ent:
        textoEntidade=entidade[0]
        labelEntidade=dicLabels[entidade[3]]
        labelEntidadeTrue='0'
        #print(textoEntidade)
        tem=0
        for entTrue in listaTrue:
            textoEntidadeTrue=entTrue[0]
            if textoEntidade == textoEntidadeTrue:
                #print('temmm:', textoEntidade)
                labelEntidadeTrue = dicLabels[entTrue[2]]
                tem=1
                break
        if tem==1:
            if labelEntidade == labelEntidadeTrue:
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
            else:
                #print('aaaaaaaaaaaaa',textoEntidade)
                #print(num)
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
        else:
            #if labelEntidade!='0':
                #print('nao temmm:{}, {}'.format(textoEntidade,labelEntidade))
                #print(num)
            regiao_true.append(labelEntidadeTrue)
            regiao_pred.append(labelEntidade)
                
    #if num>40:
    #    break
print('len(regiao_pred):', len(regiao_pred))
print('len(regiao_true):', len(regiao_true))

print('regiao_pred[:4]:', regiao_pred[:4])
print('regiao_true[:4]:', regiao_true[:4])

print(flat_classification_report(
    regiao_pred, regiao_true, labels=['1','0'], digits=3
))

agora, com pipeline... regiao CRF binario, cluster 300, janela 4 => com pipeline
len(regiao_pred): 3355
len(regiao_true): 3355
regiao_pred[:4]: ['1', '1', '0', '0']
regiao_true[:4]: ['1', '1', '0', '0']
              precision    recall  f1-score   support

           1      0.841     0.849     0.845       906
           0      0.944     0.941     0.942      2449

    accuracy                          0.916      3355
   macro avg      0.893     0.895     0.894      3355
weighted avg      0.916     0.916     0.916      3355



In [137]:
dic_predictions[15]

[[['calcificação', 0],
  ['mitral', 1],
  ['e', 2],
  ['aórtica', 3],
  ['com', 4],
  ['refluxo', 5],
  ['leve', 6],
  ['.', 7]],
 [['calcificação', [0], 'Problema'],
  ['mitral', [1], 'Tratamento'],
  ['e', [2], 'Problema'],
  ['aórtica', [3], 'Anatomia'],
  ['refluxo leve', [5, 6], 'Problema']]]

In [156]:
listacombinacoesTestPred[13]

[['calcificação', [], ['mitral', 'e', 'aórtica', 'com'], 'ENT'],
 ['mitral', ['calcificação'], ['e', 'aórtica', 'com', 'refluxo'], 'ENT'],
 ['e',
  ['calcificação', 'mitral'],
  ['aórtica', 'com', 'refluxo', 'leve'],
  'ENT'],
 ['aórtica',
  ['calcificação', 'mitral', 'e'],
  ['com', 'refluxo', 'leve', '.'],
  'ENT'],
 ['refluxo leve', ['mitral', 'e', 'aórtica', 'com'], ['.'], 'ENT'],
 ['refluxo', ['mitral', 'e', 'aórtica', 'com'], ['leve', '.'], 'O'],
 ['leve', ['e', 'aórtica', 'com', 'refluxo'], ['.'], 'O']]

In [157]:
y_pred[13]

['O', 'O', 'O', 'O', 'O', 'O', 'O']

In [203]:
dicPred

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 6,
 5: 7,
 6: 8,
 7: 9,
 8: 10,
 9: 11,
 10: 12,
 11: 13,
 12: 14,
 13: 15,
 14: 16,
 15: 17,
 16: 18,
 17: 19,
 18: 20,
 19: 23,
 20: 24,
 21: 25,
 22: 26,
 23: 27,
 24: 28,
 25: 30,
 26: 31,
 27: 32,
 28: 33,
 29: 34,
 30: 35,
 31: 36,
 32: 37,
 33: 38,
 34: 41,
 35: 42,
 36: 43,
 37: 44,
 38: 45,
 39: 46,
 40: 47,
 41: 48,
 42: 49,
 43: 50,
 44: 51,
 45: 52,
 46: 53,
 47: 54,
 48: 56,
 49: 58,
 50: 59,
 51: 60,
 52: 61,
 53: 62,
 54: 63,
 55: 64,
 56: 65,
 57: 66,
 58: 67,
 59: 68,
 60: 69,
 61: 70,
 62: 71,
 63: 72,
 64: 74,
 65: 75,
 66: 76,
 67: 77,
 68: 78,
 69: 79,
 70: 80,
 71: 82,
 72: 83,
 73: 84,
 74: 85,
 75: 86,
 76: 87,
 77: 88,
 78: 89,
 79: 90,
 80: 93,
 81: 94,
 82: 95,
 83: 97,
 84: 98,
 85: 99,
 86: 101,
 87: 103,
 88: 104,
 89: 105,
 90: 106,
 91: 107,
 92: 109,
 93: 110,
 94: 111,
 95: 112,
 96: 113,
 97: 114,
 98: 115,
 99: 116,
 100: 117,
 101: 118,
 102: 119,
 103: 121,
 104: 123,
 105: 124,
 106: 130,
 107: 131,
 108: 132,
 109:

In [226]:
#formato:
#dicPred
listaPredictRegiao = list()
listaPredictFrases = list()
#dicPred2={}
num=0
for labels, regioes in zip(y_pred, listacombinacoesTestPred):
    #print('labels:', labels)
    #print('regioes:', regioes)
    #print('num:', num)
    numKey=dicPred[num]
    #listaTrue=dic_sentencesTest[numKey][1]
    allTokens = dic_sentencesTest[numKey][0]
    #allTokens = dic_sentencesTest[num][0]
    frase=' '.join([a[0] for a in allTokens])
    #print('frase:', frase)
    for label, regiao in zip(labels, regioes):
        #print(label)
        #print(regiao)
        #listaPredictRegiao.append([regiao, frase, indices, label])
        if label == 'ENT':
            listaPredictRegiao.append([regiao[0], frase])
    #if len(listaPredictRegiao)>0:
    #    listaPredictFrases.append(listaPredictRegiao)
    #    listaPredictRegiao = list()
    listaPredictFrases.append(listaPredictRegiao)
    listaPredictRegiao = list()
    #else:
    #    print('naooo tem num:', num)
    num=num+1
    #dicPred2[num] = numKey
    
print(listaPredictFrases[13])
#dicPred2


[]


In [234]:
listaPredictFrases[12:15]

[[['aumento moderado de átrio esquerdo',
   'aumento moderado de átrio esquerdo .']],
 [],
 [['FA', 'A : FA .']]]

# Agora,  pipeline completo

In [42]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
import pandas as pd
import logging

print('-----Avaliando só com modelo de Sentence Pairs:-----')
model = ClassificationModel('bert', 'lisaterumi/sentence_pairs_nested_positivos', use_cuda=False)


-----Avaliando só com modelo de Sentence Pairs:-----


In [43]:
predictions_teste, _ = model.predict([['Comorbidades', 'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'], ['DM', 'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'], ['metformina 850mg', 'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'], ['acarbose', 'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'], ['glicazida 60mg', 'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'], ['insulina', 'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .']]) 
predictions_teste

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([0, 0, 1, 1, 1, 1], dtype=int64)

In [114]:
predictions_teste, _ = model.predict([['PA',
  'PA - 120 / 80 , P - 72 , bulhas arritimicas , normofonéticas , sem sopros .'],
 ['P',
  'PA - 120 / 80 , P - 72 , bulhas arritimicas , normofonéticas , sem sopros .'],
 ['sopros',
  'PA - 120 / 80 , P - 72 , bulhas arritimicas , normofonéticas , sem sopros .']]) 
predictions_teste

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([2, 2, 0], dtype=int64)

In [ ]:
#%%capture 
# para suprimir output
#combinacaoEntidadesAll = f.load_obj('combinacaoEntidadesAll_'+str(BATCH))
#BATCH=506
BATCH=506
print('Chamando predict')
pred_region_labels = list()
for key, combinacao in enumerate(listaPredictFrases):
    if key<BATCH:    
        if len(combinacao)>0:
            lista = [l[0:2] for l in combinacao]
            print('lista:', lista)
            if len(lista)>0:
                predictions, _ = model.predict(lista) 
            else:
                predictions=[]
            print('predictions:', predictions)
            pred_region_labels.append([p for p in predictions])
        else:
            predictions=[]
            print('combinacao vazia, key: ', key)
            pred_region_labels.append([p for p in predictions])
            #for comb, label in zip(combinacao, predictions):
            #    comb.append(label)

Chamando predict
lista: [['FA', 'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .'], ['marevan 5mg', 'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .']]


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

predictions: [0 1]
lista: [['Comorbidades', 'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'], ['DM', 'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'], ['metformina 850mg', 'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'], ['acarbose', 'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'], ['glicazida 60mg', 'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'], ['insulina', 'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

predictions: [0 1 1 1 1]
lista: [['DSLP', 'DSLP em uso de sinvastatina , marevan 1 cp / dia seg - sab para no alvo sic .'], ['sinvastatina', 'DSLP em uso de sinvastatina , marevan 1 cp / dia seg - sab para no alvo sic .'], ['marevan', 'DSLP em uso de sinvastatina , marevan 1 cp / dia seg - sab para no alvo sic .']]


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [256]:
len(pred_region_labels)

20

In [257]:
len(listaPredictFrases)

411

In [258]:
print(pred_region_labels[13])
print(listaPredictFrases[13])

[]
[]


In [259]:
pred_region_labels[14]

[0]

In [260]:
listaPredictFrases[14]

[['FA', 'A : FA .']]

In [261]:
listaPredictFrases

[[['FA',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .'],
  ['marevan 5mg',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .']],
 [['Comorbidades',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['DM',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['metformina 850mg',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['acarbose',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['glicazida 60mg',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / di

In [262]:
dic_predictions[15]

[[['calcificação', 0],
  ['mitral', 1],
  ['e', 2],
  ['aórtica', 3],
  ['com', 4],
  ['refluxo', 5],
  ['leve', 6],
  ['.', 7]],
 [['calcificação', [0], 'Problema'],
  ['mitral', [1], 'Tratamento'],
  ['e', [2], 'Problema'],
  ['aórtica', [3], 'Anatomia'],
  ['refluxo leve', [5, 6], 'Problema']]]

In [263]:
dic_sentencesTest[17]

[[['PA', 0, 1142],
  ['-', 1, 1145],
  ['120', 2, 1147],
  ['/', 3, 1150],
  ['80', 4, 1151],
  [',', 5, 1153],
  ['P', 6, 1155],
  ['-', 7, 1157],
  ['72', 8, 1159],
  [',', 9, 1161],
  ['bulhas', 10, 1163],
  ['arritimicas', 11, 1170],
  [',', 12, 1181],
  ['normofonéticas', 13, 1183],
  [',', 14, 1197],
  ['sem', 15, 1199],
  ['sopros', 16, 1203],
  ['.', 17, 1209]],
 [['PA', [0], 'Teste'],
  ['P', [6], 'Teste'],
  ['bulhas arritimicas', [10, 11], 'Problema'],
  ['sopros', [16], 'Problema']]]

In [264]:
len(dic_sentencesTest)

506

In [285]:
regiao_true=list()
regiao_pred=list()

getLabels=['Problema', 'Tratamento', 'Teste', 'Anatomia']

#dicLabels={'O':'0', 'ENT':'1', 'Problema':'1', 'Tratamento':'1', 'Teste':'1', 'Anatomia':'1'}
#dicLabels={'O':0, 'ENT':1, 'Problema':1, 'Tratamento':1, 'Teste':1, 'Anatomia':1}
dicLabels={'Problema':'0', 'Tratamento':'1', 'Teste':'2', 'Anatomia':'3'}
#dicLabels={0:'Problema', 1:'Tratamento', 2:'Teste', 3:'Anatomia'}
#labels = {0:'Problema', 1:'Tratamento', 2:'Teste', 3:'Anatomia'}

num=0
for ent, label in zip(listaPredictFrases, pred_region_labels):
    #print('--ent--', ent)
    #print('label:', label)
    numKey=dicPred[num]
    listaTrue=dic_sentencesTest[numKey][1]
    num=num+1
    if num<BATCH:    
        #print('num:', num)
        #print('listaTrue:', listaTrue)
        #print('ent:',ent)
        for numEnt, entidade in enumerate(ent):
            #print('entidade:', entidade)
            #print('numEnt:', numEnt)
            #print('label[numEnt]:', label[numEnt])
            textoEntidade=entidade[0]
            labelEntidade=str(label[numEnt])
            #labelEntidade=dicLabels[label[numEnt]]
            #labelEntidade='0'
            #labelEntidadeTrue='Problema'
            labelEntidadeTrue='0'
            
            tem=0
            '''
            print('listaTrue[numEnt]:', listaTrue[numEnt])
            if listaTrue[numEnt][0] == textoEntidade:
                #tem=1
                labelEntidadeTrue = dicLabels[listaTrue[numEnt][2]]
            print('labelEntidadeTrue:', labelEntidadeTrue)
            print('labelEntidade:', labelEntidade)
            regiao_true.append(labelEntidadeTrue)
            regiao_pred.append(labelEntidade)
            '''

            for entTrue in listaTrue:
                textoEntidadeTrue=entTrue[0]
                #print('textoEntidadeTrue:', textoEntidadeTrue)
                if textoEntidade == textoEntidadeTrue:
                    #print('temmm:', textoEntidade)
                    tem=1
                    labelEntidadeTrue = dicLabels[entTrue[2]]
                    #labelEntidadeTrue = entTrue[2]
                    regiao_true.append(labelEntidadeTrue)
                    regiao_pred.append(labelEntidade)
            if tem==0:
                print('nao achou:', textoEntidade)
                print('labelEntidadeTrue:', labelEntidadeTrue)
                print('labelEntidade:', labelEntidade)
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
                
       
                
    #if num>40:
    #    break
print('len(regiao_pred):', len(regiao_pred))
print('len(regiao_true):', len(regiao_true))

print('regiao_pred[:4]:', regiao_pred[:4])
print('regiao_true[:4]:', regiao_true[:4])

print(flat_classification_report(
    regiao_pred, regiao_true, labels=['0','1','2','3'], digits=3
    #regiao_pred, regiao_true, labels=getLabels, digits=3
))

nao achou: concentrica de grau discreto
labelEntidadeTrue: 0
labelEntidade: 0
nao achou: nos MMII
labelEntidadeTrue: 0
labelEntidade: 3
nao achou: dose da sinvastatina para 40mg
labelEntidadeTrue: 0
labelEntidade: 1
len(regiao_pred): 53
len(regiao_true): 53
regiao_pred[:4]: ['0', '1', '0', '0']
regiao_true[:4]: ['0', '1', '0', '0']
              precision    recall  f1-score   support

           0      0.909     1.000     0.952        20
           1      1.000     0.917     0.957        12
           2      1.000     1.000     1.000        17
           3      1.000     0.750     0.857         4

    accuracy                          0.962        53
   macro avg      0.977     0.917     0.942        53
weighted avg      0.966     0.962     0.961        53



In [272]:
listaPredictFrases

[[['FA',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .'],
  ['marevan 5mg',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .']],
 [['Comorbidades',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['DM',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['metformina 850mg',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['acarbose',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['glicazida 60mg',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / di

In [188]:
print('-----Avaliando modelo sem pipeline:-----')

region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_sentence, BATCH)

-----Avaliando modelo sem pipeline:-----


NameError: name 'dic_predictions_sentence' is not defined

In [124]:
dicSentences_new_test[2]

[[['Comorbidades', 0, 142],
  [':', 1, 154],
  ['DM', 2, 156],
  ['há', 3, 159],
  ['10', 4, 162],
  ['anos', 5, 165],
  ['em', 6, 170],
  ['uso', 7, 173],
  ['de', 8, 177],
  ['metformina', 9, 180],
  ['850mg', 10, 191],
  ['3', 11, 197],
  ['cp', 12, 199],
  ['/', 13, 201],
  ['dia', 14, 202],
  [',', 15, 205],
  ['acarbose', 16, 207],
  ['1', 17, 216],
  ['cp', 18, 218],
  ['/', 19, 220],
  ['dia', 20, 221],
  ['e', 21, 225],
  ['glicazida', 22, 227],
  ['60mg', 23, 237],
  ['2', 24, 242],
  ['cp', 25, 244],
  ['/', 26, 246],
  ['dia', 27, 247],
  ['e', 28, 251],
  ['insulina', 29, 253],
  ['(', 30, 262],
  ['24', 31, 263],
  ['-', 32, 266],
  ['0', 33, 268],
  ['-', 34, 270],
  ['24', 35, 272],
  [')', 36, 274],
  ['.', 37, 275]],
 [['Comorbidades', [0], 'Problema'],
  ['DM', [2], 'Problema'],
  ['metformina 850mg', [9, 10], 'Tratamento'],
  ['acarbose', [16], 'Tratamento'],
  ['glicazida 60mg', [22, 23], 'Tratamento'],
  ['insulina', [29], 'Tratamento']]]

# ========================

In [115]:
example_sent = testdata[18]
print("\nSentence:", ' '.join(sent2tokens(example_sent)))
print(' '.join(sent2tokens(example_sent)))
print("Predicted:", ' '.join(crf.predict([sent2features(example_sent)])[0]))
print("Correct:  ", ' '.join(sent2labels(example_sent)))


NameError: name 'testdata' is not defined

In [27]:
with open('predicts_crf_1nivel.csv','w') as arq:
    for i in range(0,len(testdata)):
        example_sent = testdata[i]
        #arq.write("\n\nSentence:"+ ' '.join(sent2tokens(example_sent)))
        #arq.write("\nPredicted:"+ ' '.join(crf.predict([sent2features(example_sent)])[0]))
        #arq.write("\nCorrect:  "+ ' '.join(sent2labels(example_sent)))
        arq.write('"')
        arq.write('","'.join(sent2tokens(example_sent)))
        arq.write('"\n"')
        arq.write('","'.join(crf.predict([sent2features(example_sent)])[0]))
        arq.write('"\n"')
        arq.write('","'.join(sent2labels(example_sent)))
        arq.write('"\n\n')
        #print("\nSentence:", ' '.join(sent2tokens(example_sent)))
        #print("Predicted:", ' '.join(crf.predict([sent2features(example_sent)])[0]))
        #print("Correct:  ", ' '.join(sent2labels(example_sent)))


In [37]:
for i in range(0,5):
    example_sent = testdata[i]
    print("\nSentence:", ' '.join(sent2tokens(example_sent)))
    print("Predicted:", ' '.join(crf.predict([sent2features(example_sent)])[0]))
    print("Correct:  ", ' '.join(sent2labels(example_sent)))



Sentence: Lucas , 74 anos .
Predicted: O O O O O
Correct:   O O O O O

Sentence: Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .
Predicted: O O O O O O O O Problema O O O Tratamento Tratamento O O O O O
Correct:   O O O O O O O O Problema O O O Tratamento Tratamento O O O O O

Sentence: Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .
Predicted: Problema O Problema O O O O O O Tratamento Tratamento O O O O O O O O O O O Tratamento Tratamento O O O O O Tratamento O O O O O O O O
Correct:   Problema O Problema O O O O O O Tratamento Tratamento O O O O O Tratamento O O O O O Tratamento Tratamento O O O O O Tratamento O O O O O O O O

Sentence: HAS há 15 anos em uso de losartana 50mg / dia e digoxina 1 / 2 cp / dia , carvedilol 25 12 / 12 , HCTZ .
Predicted: Problema O O O O O O Tratamento Tratamento O O O Teste O O O O O O O Tratamento Tratamento O O O O O 